pip install torch torchvision torchaudio
pip install opencv-python
pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html

In [1]:
import re
import fitz
import spacy 
from spacy.matcher import Matcher
from  transformers import LayoutLMForTokenClassification, LayoutLMTokenizer
import torch

In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cpu
False


In [3]:
def limpiar_texto(texto):
    """Normaliza y limpia el texto para mejorar la extracción de metadatos."""
    texto = re.sub(r'\s+', ' ', texto)  # Remueve espacios extra
    texto = re.sub(r'[^\x20-\x7E\xC0-\xFF]', '', texto)  # Remueve caracteres especiales
    return texto.strip()

def leer_pdf(ruta_pdf):
    """Lee un archivo PDF y devuelve su contenido en texto limpio."""
    documento = fitz.open(ruta_pdf)
    texto = ""
    for pagina in documento:
        texto += pagina.get_text("text") + "\n"
    return limpiar_texto(texto)

In [4]:
# Cargar modelos de NLP
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    from spacy.cli import download
    download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

tokenizer = LayoutLMTokenizer.from_pretrained('microsoft/layoutlm-base-uncased')
model = LayoutLMForTokenClassification.from_pretrained('microsoft/layoutlm-base-uncased', num_labels=13)

def postprocess_with_spacy(text, predicted_labels):
    """Postprocesa el texto y las etiquetas predichas usando spaCy."""
    doc = nlp(text)
    entities = {}
    
    for token, label in zip(doc, predicted_labels):
        if label != "O":  # Ignorar tokens fuera de entidades
            if label not in entities:
                entities[label] = []
            entities[label].append(token.text)
    
    # Unir tokens en frases completas
    for key in entities:
        entities[key] = " ".join(entities[key])
    
    return entities
    

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:

def extraer_metadata(pdf_path):
    """Extrae la metadata estructurada en JSON desde un PDF."""
    texto = leer_pdf(pdf_path)
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs).logits
    predictions = torch.argmax(outputs, dim=-1).squeeze().tolist()
    labels = ["O", "ISSN", "FECHA", "ENTIDAD", "AÑO_ROMANO", "NUMERO_PUBLICACION", "DIRECTOR_SENADO_NOMBRE", "DIRECTOR_SENADO_CARGO", "DIRECTOR_CAMARA_NOMBRE", "DIRECTOR_CAMARA_CARGO", "TIPO_DOCUMENTO", "SUBTITULO", "SUBSUBTITULO"]
    predicted_labels = [labels[p] for p in predictions]
    metadata = postprocess_with_spacy(texto, predicted_labels)
    
    # Estructurar en JSON validado
    json_output = {
        "titulo": "Gaceta del Congreso",
        "Entidaes": "SENADO Y CAMARA",
        "Articulo": "(Articulo 36,  Ley 5a de 1992)",
        "issn": metadata.get("ISSN", "No encontrado"),
        "fecha": metadata.get("FECHA", "No encontrado"),
        "entidad": metadata.get("ENTIDAD", "No encontrado"),
        "numero_publicacion": metadata.get("NUMERO_PUBLICACION", "No encontrado"),
        "directores": {
            "senado": {
                "nombre": metadata.get("DIRECTOR_SENADO_NOMBRE", "No encontrado"),
                "cargo": metadata.get("DIRECTOR_SENADO_CARGO", "No encontrado")
            },
            "camara": {
                "nombre": metadata.get("DIRECTOR_CAMARA_NOMBRE", "No encontrado"),
                "cargo": metadata.get("DIRECTOR_CAMARA_CARGO", "No encontrado")
            }
        },
        "tipo_documento": metadata.get("TIPO_DOCUMENTO", "No encontrado"),
        "subtitulo": metadata.get("SUBTITULO", "No encontrado"),
        "subsubtitulo": metadata.get("SUBSUBTITULO", "No encontrado")
    }
    
    return json_output


In [6]:
ruta_gaceta = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64_removed.pdf"
resultado = extraer_metadata(ruta_gaceta)
print(resultado)

FileNotFoundError: no such file: 'C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328_XXV_110_64_removed.pdf'